# Notebook to demonstrate multiscale cleaning on the canonical "M31" source.

We simulate some VLA data for M31 and then use msclean to restore it.

Multiscale CLEAN, (IEEE Journal of Selected Topics in Sig Proc, 2008 vol. 2 pp. 793-801)

In [ ]:
%matplotlib inline

import sys, os 
sys.path.append(os.path.join('..','..')) 

import pylab
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

from timeit import default_timer as timer

import numpy
import scipy
import scipy.special

from matplotlib import pylab
from matplotlib import pyplot as plt

import pyfits

from crocodile.synthesis import *
from crocodile.simulate import *
from util.visualize import *
from crocodile.msclean import *

Now generate and plot the uvw coordinates

In [ ]:
vlas=numpy.genfromtxt("../../data/vis/VLA_A_hor_xyz.txt", delimiter=",") 
vlas_xyz = xyz_at_latitude(vlas, numpy.pi*34.0784/180.0)/30.0 
uvw=xyz_to_baselines(vlas_xyz, numpy.arange(-numpy.pi/2.0, +numpy.pi/2.0,0.05), numpy.pi/10)
plt.clf()
plt.plot(uvw[:,0],uvw[:,1], '.')
plt.plot(-uvw[:,0],-uvw[:,1], '.', color='red')
plt.xlabel('U (m)')
plt.ylabel('V (m)')
plt.title('UV coverage')

Next get the M31 model from the crocodile data directory

In [ ]:
m31model='../../data/models/M31.MOD'
hdulist = pyfits.open(m31model)
m31image=numpy.array(hdulist[0].data)
plt.clf()
plt.imshow(m31image, cmap='rainbow', origin='lower')
plt.show()

Calculate the visibility for m31 using wslicfwd

In [ ]:
lam=4.0*numpy.abs(uvw).max()
cell=1.0/lam
theta=256*cell
wstep=100

print("Field of view = %.2f degrees, maximum baseline %.3f (m)" % (theta*180.0/numpy.pi, lam))

The prediction and imaging steps use a least-recently-used cache of the w convolution kernels.
Here we set up the cache and pre-fill it

In [ ]:
wcachesize=int(numpy.ceil(numpy.abs(uvw[:,2]).max()/wstep))
print("Making w-kernel cache of %d kernels" % (wcachesize))
wcache=pylru.FunctionCacheManager(lambda iw: wkernaf(N=256, theta=theta, w=iw*wstep, s=15, Qpx=4), 10000)

start = timer()
for iw in range(-wcachesize,wcachesize):
    wg=wcache(iw)
mid = timer()

print("First call took %.3f (s)" % (mid-start))

for iw in range(-wcachesize,wcachesize):
    wg=wcache(iw)

end = timer()
print("Second call took %.3f (s)" % (end-mid))

Now we can define the prediction and imaging functions that will use this cache

In [ ]:
predfn=lambda *x: wcachefwd(*x, wstep=wstep, wcache=wcache)
imgfn=lambda *x: wcacheimg(*x, wstep=wstep, wcache=wcache)

Now we predict the visibility data using the w-kernels cache.

In [ ]:
uvw[0]

In [ ]:
uvwsort,vobs=dopredict(theta, lam, p=uvw, modelimage=m31image, predfn=predfn)

Plot the visibility as a function of uv distance

In [ ]:
print(uvwsort.shape)
print(uvwsort[:,2].shape)

print(vobs.shape)

uvdist=numpy.sqrt(uvwsort[:,0]*uvwsort[:,0]+uvwsort[:,1]*uvwsort[:,1])
plt.clf()
plt.plot(uvwsort[:,2], numpy.abs(vobs), '.')
plt.xlabel('w')
plt.ylabel('Amp Visibility')
plt.show()

Next we make the dirty image and psf to check if it looks plausible

In [ ]:
d,p,_=doimg(2.0*theta, lam, uvwsort, vobs, imgfn=imgfn)
plt.clf()
plt.subplot(1,2,1)
plt.imshow(p, cmap='rainbow', origin='lower')
plt.title('PSF')
c=plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(d, cmap='rainbow', origin='lower')
plt.title('Dirty image')
c=plt.colorbar()



Now we can do the deconvolution

In [ ]:
scales=[0, 3, 10, 30]
ps, vs, m31cc, m31res=majorcycle(2.0*theta, lam, uvwsort, vobs, 0.75, 3, 1000, scales, 0.20, 0.1, \
                                 predfn=predfn, \
                                 imgfn=imgfn)

Finally show the results

In [ ]:
print("Total flux in model %.6f Jy" % numpy.sum(m31cc))
plt.clf()
plt.subplot(1,2,1)
plt.imshow(m31cc, cmap='rainbow', origin='lower')
c=plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(m31res, cmap='rainbow', origin='lower')
c=plt.colorbar()




Now plot the residual visibility to see if it is noise-like

In [ ]:
plt.clf()
plt.plot(uvdist, numpy.abs(vs), '.')
plt.xlabel('UV distance (wavelengths)')
plt.ylabel('Amp Residual Visibility')
plt.show()

Plot against w to see if the errors are more easily seen

In [ ]:
plt.clf()
plt.plot(ps[:,2], numpy.abs(vs), '.')
plt.xlabel('w')
plt.ylabel('Amp Residual Visibility')
plt.show()